In [1]:
import csv
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, LSTM, Conv1D, Flatten
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import tensorflow as tf

# 1. Utworzenie zbioru treningowego i testowego

## 1.1 Otwarcie pliku

In [2]:
with open('data.csv', newline='') as f:
    reader = csv.reader(f)
    data_temp = list(reader)
data_temp.pop(0)
print(data_temp[:10])

[['0_650', '0', '650', '-1.0813170675566173', '0.013333333333333336', 'False', 'False'], ['0_675', '0', '675', '-1.0813170675566173', '0.013333333333333336', 'False', 'False'], ['0_700', '0', '700', '-1.0813170675566173', '0.013333333333333336', 'False', 'False'], ['0_725', '0', '725', '-0.9937692595422224', '0.026666666666666672', 'False', 'False'], ['0_1600', '0', '1600', '-1.0813170675566173', '0.013333333333333336', 'False', 'True'], ['0_1625', '0', '1625', '-0.9937692595422224', '0.026666666666666672', 'False', 'False'], ['0_1650', '0', '1650', '-1.0813170675566173', '0.013333333333333336', 'False', 'False'], ['0_1675', '0', '1675', '-1.0813170675566173', '0.013333333333333336', 'False', 'False'], ['0_1700', '0', '1700', '-1.1688648755710123', '0.0', 'False', 'False'], ['0_1725', '0', '1725', '-1.1688648755710123', '0.0', 'False', 'False']]


In [3]:
data = []
true = false = 0
for x in data_temp:
    if x[6] == 'True':
        true += 1
    else:
        false += 1
    data.append([x[0], float(x[3]), float(x[4]), x[5] == 'True', x[6] == 'True'])
print(data[0], true, false)

['0_650', -1.0813170675566173, 0.013333333333333336, False, False] 1374 979


## 1.2 Obliczenie rozmiaru

In [4]:
train_count = int(len(data) * 0.8)
test_count = len(data) - train_count
print(train_count + test_count, len(data))
assert train_count + test_count == len(data)

frame_size = 25

2353 2353


## 1.3 Utworzenie zbioru

Dla każdego Y dodawane jest do X frame_size wartości sensora przed

In [5]:
X_train_min_max = []
X_train_z_score = []
Y_train = []
X_test_min_max = []
X_test_z_score = []
Y_test = []

for i in range(9, train_count):
    ground_truth = data[i][4]
    Y_train.append(ground_truth)
    sensor_values_min_max = [data[j][2] for j in range(i - 9, i + 1)]
    sensor_values_z_score = [data[j][1] for j in range(i - 9, i + 1)]
    X_train_min_max.append(sensor_values_min_max)
    X_train_z_score.append(sensor_values_z_score)

for i in range(9 + train_count, len(data)):
    ground_truth = data[i][4]
    Y_test.append(ground_truth)
    sensor_values_min_max = [data[j][2] for j in range(i - 9, i + 1)]
    sensor_values_z_score = [data[j][1] for j in range(i - 9, i + 1)]
    X_test_min_max.append(sensor_values_min_max)
    X_test_z_score.append(sensor_values_z_score)

X_train_min_max = np.array(X_train_min_max)
X_train_z_score = np.array(X_train_z_score)
Y_train = np.array(Y_train)
X_test_min_max = np.array(X_test_min_max)
X_test_z_score = np.array(X_test_z_score)
Y_test = np.array(Y_test)

## Definicja modelu

In [6]:
model = Sequential()
model.add(LSTM(32, input_shape=(10, 1), dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                4352      
                                                                 
 dense (Dense)               (None, 128)               4224      
                                                                 
 dense_1 (Dense)             (None, 16)                2064      
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 10,657
Trainable params: 10,657
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(X_train_min_max, Y_train, validation_data=(X_test_min_max, Y_test), epochs=30)

Epoch 1/30
59/59 [==============================] - 19s 17ms/step - loss: 0.5715 - binary_accuracy: 0.6903 - precision: 0.6635 - recall: 0.8886 - val_loss: 0.6154 - val_binary_accuracy: 0.7186 - val_precision: 0.7186 - val_recall: 1.0000
Epoch 2/30
59/59 [==============================] - 0s 8ms/step - loss: 0.4356 - binary_accuracy: 0.8254 - precision: 0.8819 - recall: 0.7888 - val_loss: 0.8171 - val_binary_accuracy: 0.7186 - val_precision: 0.7186 - val_recall: 1.0000
Epoch 3/30
59/59 [==============================] - 0s 8ms/step - loss: 0.4284 - binary_accuracy: 0.8265 - precision: 0.8741 - recall: 0.8004 - val_loss: 0.7429 - val_binary_accuracy: 0.7186 - val_precision: 0.7186 - val_recall: 1.0000
Epoch 4/30
59/59 [==============================] - 0s 8ms/step - loss: 0.4204 - binary_accuracy: 0.8233 - precision: 0.8725 - recall: 0.7955 - val_loss: 0.7372 - val_binary_accuracy: 0.7186 - val_precision: 0.7186 - val_recall: 1.0000
Epoch 5/30
59/59 [==============================] - 0s

In [10]:
model2 = Sequential()
model2.add(Conv1D(64, 3, input_shape=(10, 1)))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [11]:
model2.fit(X_train_min_max, Y_train, validation_data=(X_test_min_max, Y_test), epochs=30)

Epoch 1/30
59/59 [==============================] - 2s 16ms/step - loss: 0.4917 - binary_accuracy: 0.8083 - precision_2: 0.8335 - recall_2: 0.8149 - val_loss: 0.6080 - val_binary_accuracy: 0.7186 - val_precision_2: 0.7186 - val_recall_2: 1.0000
Epoch 2/30
59/59 [==============================] - 1s 11ms/step - loss: 0.4130 - binary_accuracy: 0.8292 - precision_2: 0.8732 - recall_2: 0.8072 - val_loss: 0.6383 - val_binary_accuracy: 0.7186 - val_precision_2: 0.7186 - val_recall_2: 1.0000
Epoch 3/30
59/59 [==============================] - 1s 11ms/step - loss: 0.4007 - binary_accuracy: 0.8275 - precision_2: 0.8736 - recall_2: 0.8033 - val_loss: 0.7338 - val_binary_accuracy: 0.7186 - val_precision_2: 0.7186 - val_recall_2: 1.0000
Epoch 4/30
59/59 [==============================] - 1s 11ms/step - loss: 0.3915 - binary_accuracy: 0.8275 - precision_2: 0.8751 - recall_2: 0.8014 - val_loss: 0.7069 - val_binary_accuracy: 0.6320 - val_precision_2: 0.7166 - val_recall_2: 0.8072
Epoch 5/30
59/59 [==